In [ ]:
import numpy as np
import nidaqmx

In [ ]:
# Define the input and output channels
input_channel = "PXI1Slot2/ai0"
output_channels = ["PXI1Slot3/ao0", "PXI1Slot3/ao2", "PXI1Slot3/ao4", "PXI1Slot3/ao6"]

# Define the SPGD parameters
scale = 0.4
output_scale=1
lr = 0.1
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8

In [ ]:
def check_range(v,scale):
    idx = np.where(np.abs(v) + scale > 9.5)
    if idx[0].size > 0:
        v[idx[0]] = 5
    return v

In [ ]:
# %%

# %%
with nidaqmx.Task() as input_task, nidaqmx.Task() as output_task:
    
    # Configure the input task
    input_task.ai_channels.add_ai_voltage_chan(input_channel)

    # Configure the output task
    output_task.ao_channels.add_ao_voltage_chan(",".join(output_channels))
    V = 2*np.ones(4)

    # Initialize the Adam parameters
    m = np.zeros_like(V)
    v = np.zeros_like(V)
    t = 1

    for idx in range(40):

        delta_v = np.random.rand(4)

        V = check_range(V,scale)

        # print('reading..')
        output_task.write(V,auto_start=True)
        output_task.wait_until_done()
        input_signal_v = output_scale*input_task.read()

        # print('positive perturbation')
        pos_v =V+delta_v
        output_task.write(pos_v,auto_start=True)
        output_task.wait_until_done()
        input_signal_plus = output_scale*input_task.read()

        # print('negative perturbation')
        neg_v = V-delta_v
        output_task.write(neg_v,auto_start=True)
        output_task.wait_until_done()
        input_signal_minus = output_scale*input_task.read()

        gradient = (input_signal_plus - input_signal_minus) * delta_v

        # Update the Adam parameters
        t += 1
        m = beta1*m + (1-beta1)*gradient
        v = beta2*v + (1-beta2)*(gradient**2)
        m_hat = m / (1-beta1**t)
        v_hat = v / (1-beta2**t)
        # Update the voltage
        V += lr*m_hat / (np.sqrt(v_hat) + epsilon)
        # if t%100 == 0:
        print('grad:',gradient.mean(), 'v_out:',V)